# 03 - Buildings EDA (Microsoft & Google) - Local files

Proyecto: PDET Solar Rooftop Analysis

Este notebook realiza el Análisis Exploratorio de Datos (EDA) para los datasets de **Microsoft Building Footprints** y **Google Open Buildings**
usando archivos locales. No requiere conexión a MongoDB.

Rutas configuradas para este entorno Windows (ajusta si es necesario):
- Microsoft GeoJSONL: `C:\\Users\\ThinkPad X1\\Desktop\\Microsoft Datos\\Colombia.geojsonl\\Colombia.geojsonl`
- Google CSV (gz): `C:\\Users\\ThinkPad X1\\Desktop\\Google Datos\\open_buildings_v3_polygons_ne_110m_COL.csv.gz`
- PDET CSV: `data/processed/pdet_municipalities_list.csv`


## 0. Recomendaciones
- Ejecuta este notebook en un equipo con suficiente RAM si procesas todos los registros. Para pruebas, activa `SAMPLE_MODE = True`.
- Asegúrate de que las rutas arriba sean accesibles desde el entorno donde ejecutas Jupyter (Windows).

In [ ]:
%pip install --quiet geopandas pandas shapely pyarrow folium plotly

import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import shape
import plotly.express as px
import folium

os.makedirs('results/figures/deliverable_3', exist_ok=True)

# Configuración de rutas (ajusta si tu ruta es diferente)
microsoft_path = r"C:\\Users\\ThinkPad X1\\Desktop\\Microsoft Datos\\Colombia.geojsonl\\Colombia.geojsonl"
google_path = r"C:\\Users\\ThinkPad X1\\Desktop\\Google Datos\\open_buildings_v3_polygons_ne_110m_COL.csv.gz"
pdet_path = "data/processed/pdet_municipalities_list.csv"

# Modo muestra: True = cargar sólo una muestra para pruebas (rápido)
SAMPLE_MODE = True
SAMPLE_SIZE = 50000


## 1. Funciones de carga desde archivos locales
Lectores flexibles para GeoJSONL (Microsoft), CSV gz (Google) y CSV PDET.

In [ ]:
def load_microsoft_geojsonl(path, sample_mode=True, sample_size=50000):
    """Lee archivo GeoJSONL (una geometría por línea). Devuelve GeoDataFrame."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"No se encontró archivo Microsoft en: {path}")
    rows = []
    with open(path, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            rows.append(line)
            if sample_mode and i+1 >= sample_size:
                break
    import json
    features = [json.loads(r) for r in rows]
    # Si cada línea es un Feature o un FeatureCollection
    objs = []
    for feat in features:
        if 'type' in feat and feat['type'] == 'Feature':
            objs.append(feat)
        elif 'features' in feat:
            objs.extend([f for f in feat['features']])
        else:
            # intentar interpretar como GeoJSON directo
            objs.append(feat)
    # Convertir a dicts compatibles con GeoDataFrame
    geo_json = {'type': 'FeatureCollection', 'features': objs}
    gdf = gpd.GeoDataFrame.from_features(geo_json)
    if 'geometry' in gdf.columns:
        gdf = gdf.set_geometry('geometry')
        try:
            gdf = gdf.set_crs(epsg=4326, allow_override=True)
        except Exception:
            pass
    return gdf

def load_google_csv_gz(path, sample_mode=True, sample_size=50000):
    """Lee CSV comprimido (csv.gz). Convierte columna geom/wkt a geometría si existe."""
    if not os.path.exists(path):
        raise FileNotFoundError(f"No se encontró archivo Google en: {path}")
    # pandas puede leer .csv.gz directamente
    df = pd.read_csv(path, compression='gzip', low_memory=False)
    # detectar columna de geometría
    geom_col = None
    for c in ['geometry', 'geom', 'wkt', 'wkb']:
        if c in df.columns:
            geom_col = c
            break
    if geom_col:
        try:
            df['geometry'] = df[geom_col].apply(wkt.loads)
            gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
        except Exception:
            # si falla, crear GeoDataFrame sin convertir
            gdf = gpd.GeoDataFrame(df)
    else:
        gdf = gpd.GeoDataFrame(df)
    if sample_mode and len(gdf) > sample_size:
        gdf = gdf.sample(sample_size, random_state=1)
    return gdf

def load_pdet_csv(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"No se encontró archivo PDET en: {path}")
    df = pd.read_csv(path)
    # intentar detectar geometría
    if 'geometry' in df.columns:
        try:
            df['geometry'] = df['geometry'].apply(wkt.loads)
            gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')
        except Exception:
            gdf = gpd.GeoDataFrame(df)
    else:
        gdf = gpd.GeoDataFrame(df)
    return gdf


## 2. Cargar datasets locales
A continuación cargamos Microsoft, Google y PDET desde las rutas locales configuradas arriba.

In [ ]:
print('Cargando Microsoft (GeoJSONL)...')
ms_gdf = load_microsoft_geojsonl(microsoft_path, sample_mode=SAMPLE_MODE, sample_size=SAMPLE_SIZE)
print('Microsoft cargado - registros:', len(ms_gdf))

print('\nCargando Google (CSV gz)...')
ggl_gdf = load_google_csv_gz(google_path, sample_mode=SAMPLE_MODE, sample_size=SAMPLE_SIZE)
print('Google cargado - registros:', len(ggl_gdf))

print('\nCargando PDET (CSV)...')
pdet_gdf = load_pdet_csv(pdet_path)
print('PDET cargado - registros:', len(pdet_gdf))

# Mostrar columnas básicas
print('\nMicrosoft columns:', list(ms_gdf.columns)[:10])
print('Google columns:', list(ggl_gdf.columns)[:10])
print('PDET columns:', list(pdet_gdf.columns)[:10])

## 3. Resumen rápido por dataset
Funciones para mostrar conteos, geometrías inválidas y estadísticas básicas.

In [ ]:
def dataset_summary(gdf, name='dataset'):
    print(f'--- Resumen {name} ---')
    print('Registros:', len(gdf))
    print('Columnas (ejemplo):', list(gdf.columns)[:20])
    if 'geometry' in gdf.columns:
        print('Geometrías inválidas:', int((~gdf.is_valid).sum()))
    if 'area_m2' in gdf.columns:
        print('area_m2 - mean/min/max:', gdf['area_m2'].mean(), gdf['area_m2'].min(), gdf['area_m2'].max())
    if 'area_km2' in gdf.columns:
        print('area_km2 - mean/min/max:', gdf['area_km2'].mean(), gdf['area_km2'].min(), gdf['area_km2'].max())
    if 'confidence' in gdf.columns:
        print('confidence - mean/min/max:', gdf['confidence'].mean(), gdf['confidence'].min(), gdf['confidence'].max())
    print('\n')
dataset_summary(ms_gdf, 'Microsoft')
dataset_summary(ggl_gdf, 'Google')
dataset_summary(pdet_gdf, 'PDET')

## 4. Cobertura por municipio PDET (spatial join)
Se realiza un spatial join para contar edificaciones por municipio PDET. Esta operación puede ser costosa; en SAMPLE_MODE usamos muestras.

In [ ]:
def buildings_per_municipio(buildings_gdf, pdet_gdf, muni_col='municipio'):
    if len(buildings_gdf)==0 or len(pdet_gdf)==0:
        return pd.DataFrame()
    # Asegurar CRS
    if buildings_gdf.crs is None:
        buildings_gdf = buildings_gdf.set_crs(epsg=4326, allow_override=True)
    if pdet_gdf.crs is None:
        pdet_gdf = pdet_gdf.set_crs(epsg=4326, allow_override=True)
    # Si las geometrías de edificios son polígonos, podemos usar centroid para acelerar (opcional)
    try:
        pts = buildings_gdf.copy()
        if pts.geometry.geom_type.isin(['Polygon','MultiPolygon']).any():
            pts['geometry'] = pts.geometry.centroid
    except Exception:
        pass
    joined = gpd.sjoin(pts, pdet_gdf[['municipio','geometry']], how='inner', predicate='intersects')
    counts = joined.groupby('municipio').size().reset_index(name='buildings_count')
    merged = pdet_gdf.merge(counts, on='municipio', how='left')
    merged['buildings_count'] = merged['buildings_count'].fillna(0).astype(int)
    return merged[['municipio','buildings_count']]

ms_muni_counts = buildings_per_municipio(ms_gdf, pdet_gdf)
ggl_muni_counts = buildings_per_municipio(ggl_gdf, pdet_gdf)

ms_muni_counts.to_csv('results/figures/deliverable_3/ms_buildings_by_municipio_sample.csv', index=False)
ggl_muni_counts.to_csv('results/figures/deliverable_3/ggl_buildings_by_municipio_sample.csv', index=False)
print('Conteos por municipio exportados (sample)')

## 5. Comparación entre datasets
Tabla comparativa de conteos por municipio y diferencia entre Microsoft y Google (muestra).

In [ ]:
comp = ms_muni_counts.merge(ggl_muni_counts, on='municipio', how='outer', suffixes=('_ms','_ggl')).fillna(0)
comp['diff'] = comp['buildings_count_ms'] - comp['buildings_count_ggl']
comp = comp.sort_values('buildings_count_ms', ascending=False)
comp.to_csv('results/figures/deliverable_3/comp_buildings_by_municipio_sample.csv', index=False)
display(comp.head(10))

## 6. Overlap analysis (detecciones en ambos datasets)
Spatial join entre Microsoft y Google con buffer en metros (reproyectar a CRS métrico).

In [ ]:
def overlap_analysis(gdf1, gdf2, buffer_m=1.0, sample_mode=True, sample_size=20000):
    if len(gdf1)==0 or len(gdf2)==0:
        return pd.DataFrame()
    g1 = gdf1.to_crs(epsg=3857)
    g2 = gdf2.to_crs(epsg=3857)
    if sample_mode:
        g1 = g1.sample(min(len(g1), sample_size), random_state=1)
        g2 = g2.sample(min(len(g2), sample_size), random_state=1)
    g1['buf'] = g1.geometry.buffer(buffer_m)
    g1_buf = g1.set_geometry('buf')
    joined = gpd.sjoin(g1_buf, g2[['geometry']], how='inner', predicate='intersects')
    return joined

joined_overlap = overlap_analysis(ms_gdf, ggl_gdf, buffer_m=1.0, sample_mode=SAMPLE_MODE)
print('Overlap (sample) rows:', len(joined_overlap))

## 7. Visualizaciones
- Histograma de áreas (si existe campo `area_m2` o `area_km2`).
- Heatmaps de densidad usando centroides.
- Gráfica comparativa top municipios.

In [ ]:
# Histograma de áreas
if 'area_m2' in ms_gdf.columns:
    fig = px.histogram(ms_gdf, x='area_m2', nbins=100, title='Distribución áreas - Microsoft')
    fig.write_html('results/figures/deliverable_3/ms_area_histogram.html')
if 'area_m2' in ggl_gdf.columns:
    fig = px.histogram(ggl_gdf, x='area_m2', nbins=100, title='Distribución áreas - Google')
    fig.write_html('results/figures/deliverable_3/ggl_area_histogram.html')

# Heatmap helper
from folium.plugins import HeatMap
def save_heatmap(gdf, out_html, sample=20000):
    if len(gdf)==0:
        print('Sin datos para heatmap')
        return
    gg = gdf.copy()
    if gg.crs is None:
        gg = gg.set_crs(epsg=4326, allow_override=True)
    gg = gg.to_crs(epsg=4326)
    coords = list(zip(gg.geometry.centroid.y, gg.geometry.centroid.x))
    if len(coords) > sample:
        import random
        coords = random.sample(coords, sample)
    m = folium.Map(location=[4.6, -74.1], zoom_start=6, tiles='CartoDB positron')
    HeatMap(coords, radius=8, blur=10).add_to(m)
    m.save(out_html)
    print('Heatmap guardado en', out_html)

save_heatmap(ms_gdf, 'results/figures/deliverable_3/ms_heatmap.html', sample=20000)
save_heatmap(ggl_gdf, 'results/figures/deliverable_3/ggl_heatmap.html', sample=20000)

# Gráfica comparativa top municipios
topN = 20
comp_top = comp.sort_values(['buildings_count_ms'], ascending=False).head(topN)
fig = px.bar(comp_top, x='municipio', y=['buildings_count_ms','buildings_count_ggl'],
             title=f'Top {topN} municipios: Microsoft vs Google (muestra)')
fig.update_layout(barmode='group')
fig.write_html('results/figures/deliverable_3/top_municipios_comparison.html')
print('Visualizaciones guardadas en results/figures/deliverable_3/')

## 8. Guardar artefactos y conclusiones
Exportar CSVs y conclusiones preliminares.

In [ ]:
artifacts = {
    'ms_muni_counts': 'results/figures/deliverable_3/ms_buildings_by_municipio_sample.csv',
    'ggl_muni_counts': 'results/figures/deliverable_3/ggl_buildings_by_municipio_sample.csv',
    'comp': 'results/figures/deliverable_3/comp_buildings_by_municipio_sample.csv'
}
print('Artifacts paths:')
for k,v in artifacts.items():
    print('-', k, ':', v)

conclusions = '''\nPreliminary conclusions (sample):\n- Microsoft y Google presentan diferencias en conteo y cobertura por municipio.\n- Algunos municipios muestran mayor densidad en Microsoft, otros en Google.\n- El análisis de overlapping necesita full-run para mayor precisión.\n'''
with open('results/figures/deliverable_3/conclusions.txt','w',encoding='utf-8') as f:
    f.write(conclusions)
print('Conclusions written to results/figures/deliverable_3/conclusions.txt')